In [1]:
import numpy as np
import plotly.express as px
import cv2

def parse_input():
    cubes = []
    with open("input.txt","r") as f:
        for line in f:
            splt = line.rstrip().split(",")
            cubes.append((int(splt[0]), int(splt[1]), int(splt[2])))
    return cubes

cubes = parse_input()

In [2]:
max_x = max(map(lambda c: c[0], cubes))
max_y = max(map(lambda c: c[1], cubes))
max_z = max(map(lambda c: c[2], cubes))

pad = max(max_x, max_y, max_z)*2

grid = np.pad(np.zeros((max_x,max_y,max_z)),pad) # use padded grid so we do not have to do border checks in 3d space, also better for edge detection later

def get_neighbours(grid,x,y,z) -> list[tuple[tuple[int,int,int],int]]:
    n = [
        ((x+1, y, z), grid[x+1][y][z]),
        ((x-1, y, z), grid[x-1][y][z]),
        ((x, y+1, z), grid[x][y+1][z]),
        ((x, y-1, z), grid[x][y-1][z]),
        ((x, y, z+1), grid[x][y][z+1]),
        ((x, y, z-1), grid[x][y][z-1])
    ]
    return n

for (x,y,z) in cubes:
    sides = 6
    neighbours = get_neighbours(grid, x+pad, y+pad, z+pad)
    for (n_x, n_y, n_z), n_sides in neighbours:
        if n_sides > 0:
            sides = sides - 1
            grid[n_x][n_y][n_z] = max(0,grid[n_x][n_y][n_z] - 1)
    grid[x+pad][y+pad][z+pad] = max(0,sides)

pad, np.sum(grid)

(38, 3498.0)

In [3]:
plt_data =[(x,y,z,v) for (x,y,z) , v in  np.ndenumerate(grid) if v > 0]

fig = px.scatter_3d(
    x=[n[0] for n in plt_data],
    y=[n[1] for n in plt_data],
    z=[n[2] for n in plt_data],
    text=[n[3] for n in plt_data]
)

fig.update_traces(marker_size = 1)

fig

# Part 2

- use Morphological closing with open cv for each slice
- re-calculate surface of the drop. as drop is now closed, correct surface area should be calculated

In [8]:
# function to convert faces to binary pixels
filtr = np.vectorize(lambda x : 1 if x > 0 else 0)
grayscale_drop = filtr(grid)
px.imshow(grayscale_drop[40])

In [21]:
kernel = np.ones((5,5), np.uint8)
closing = cv2.morphologyEx(grayscale_drop[40].astype("uint8"), cv2.MORPH_CLOSE, kernel)
px.imshow(closing)
# px.imshow(grayscale_drop[5].astype("uint8"))



In [17]:
t = []
for slice in grayscale_drop:
    t.append(cv2.morphologyEx(slice.astype("uint8"), cv2.MORPH_CLOSE, kernel))

filled_grayscale_drop = np.array(t)

px.imshow(filled_grayscale_drop[40])

In [18]:
faces = 0
for (x,y,z) , v in  np.ndenumerate(filled_grayscale_drop):
    if v > 0 :
        neighbours = get_neighbours(filled_grayscale_drop, x, y, z)
        s = 6
        for (n_x, n_y, n_z), n_sides in neighbours:
            if n_sides > 0:
                s = s - 1
        faces = faces + s

faces


2372